# CompScholar Research Paper Summarization
## Complete Pipeline with Hybrid Section Markers


In [1]:
!pip install transformers datasets evaluate rouge-score nltk sentencepiece

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.8 MB/s eta 0:00:00
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=61d402f5142ab44fee2da7002da7e4d23a209701570119d88a4719b8ab71bb3a
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


In [2]:
import re
import numpy as np
import pandas as pd
from datasets import Dataset, DatasetDict
from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
)
import evaluate
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

## Step 1: Data Loading & Preprocessing
Enhanced cleaning and section marking

In [3]:
def preprocess_compscholar(df):
    """Full preprocessing pipeline for CompScholar data"""
    # Handle missing values
    df = df.fillna('')
    
    # Clean text components
    def clean_text(text):
        text = re.sub(r'\s+', ' ', text)  # Remove extra whitespace
        text = re.sub(r'[^\w\s.,;:!?()-]', '', text)  # Remove special chars
        return text.strip()
    
    # Build structured document with section markers
    structured_docs = []
    for _, row in df.iterrows():
        sections = [
            f"<title>{clean_text(row['Paper Title'])}</title>",
            f"<keywords>{clean_text(row['Key Words'])}</keywords>",
            f"<abstract>{clean_text(row['Abstract'])}</abstract>",
            f"<conclusion>{clean_text(row['Conclusion'])}</conclusion>"
        ]
        structured_docs.append('\n\n'.join(sections))
    
    df['structured_text'] = structured_docs
    df['clean_summary'] = df['Summary'].apply(clean_text)
    return df

# Load and preprocess dataset
full_df = pd.read_csv('/kaggle/input/compscholar/Brain Dead CompScholar Dataset.csv')
processed_df = preprocess_compscholar(full_df)

# Train/Val/Test split (80/10/10)
train_df = processed_df.sample(frac=0.8, random_state=42)
val_test_df = processed_df.drop(train_df.index)
val_df = val_test_df.sample(frac=0.5, random_state=42)
test_df = val_test_df.drop(val_df.index)

# Create Hugging Face datasets
dataset = DatasetDict({
    'train': Dataset.from_pandas(train_df),
    'validation': Dataset.from_pandas(val_df),
    'test': Dataset.from_pandas(test_df)
})

## Step 2: Tokenization with Section Awareness
Special tokens for document structure

In [4]:
model_name = 'facebook/bart-large-cnn'
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Add special tokens for section markers
new_tokens = ['<title>', '</title>', '<keywords>', '</keywords>', 
              '<abstract>', '</abstract>', '<conclusion>', '</conclusion>']
tokenizer.add_tokens(new_tokens)

max_input_length = 1024
max_target_length = 256

def tokenize_function(examples):
    """Tokenization with section structure preservation"""
    inputs = [text for text in examples['structured_text']]
    targets = [text for text in examples['clean_summary']]
    
    model_inputs = tokenizer(
        inputs,
        max_length=max_input_length,
        truncation=True,
        padding='max_length',
        add_special_tokens=True
    )
    
    labels = tokenizer(
        targets,
        max_length=max_target_length,
        truncation=True,
        padding='max_length'
    )
    
    model_inputs['labels'] = labels['input_ids']
    return model_inputs

tokenized_dataset = dataset.map(tokenize_function, batched=True)

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Map:   0%|          | 0/297 [00:00<?, ? examples/s]

Map:   0%|          | 0/37 [00:00<?, ? examples/s]

Map:   0%|          | 0/37 [00:00<?, ? examples/s]

## Step 3: Model Initialization
 BART model with extended vocabulary

In [5]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
model.resize_token_embeddings(len(tokenizer))  # For new section tokens

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


BartScaledWordEmbedding(50273, 1024, padding_idx=1)

## Step 4: Training Configuration
Optimized for small dataset

In [6]:
training_args = Seq2SeqTrainingArguments(
    output_dir='./results',
    evaluation_strategy='epoch',
    learning_rate=3e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=15,  # More epochs for small dataset
    predict_with_generate=True,
    fp16=True,
    logging_steps=100,
    warmup_ratio=0.1,
    gradient_accumulation_steps=2,
    report_to='none'
)

# Initialize metrics
rouge = evaluate.load('rouge')
bleu = evaluate.load('bleu')

def compute_metrics(eval_pred):
    """Enhanced metric calculation with length penalty"""
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # ROUGE with stemming
    rouge_result = rouge.compute(
        predictions=decoded_preds,
        references=decoded_labels,
        use_stemmer=True,
        use_aggregator=True
    )
    
    # BLEU with smoothing
    bleu_result = bleu.compute(
        predictions=decoded_preds,
        references=[[ref] for ref in decoded_labels],
        max_order=4,
        smooth=True
    )
    
    return {
        'rouge1': round(rouge_result['rouge1'], 3),
        'rouge2': round(rouge_result['rouge2'], 3),
        'rougeL': round(rouge_result['rougeL'], 3),
        'bleu': round(bleu_result['bleu'], 3)
    }



/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


## Step 5: Model Training
With early stopping

In [7]:

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['validation'],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()

<ipython-input-7-f1ac3738ac86>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Bleu
1,No log,1.068682,0.610000,0.352000,0.446000,0.274000
2,No log,0.521526,0.609000,0.348000,0.442000,0.288000
3,1.928700,0.516122,0.616000,0.363000,0.452000,0.290000
4,1.928700,0.532636,0.597000,0.339000,0.431000,0.273000
5,1.928700,0.621746,0.615000,0.359000,0.453000,0.290000
6,0.199500,0.671185,0.623000,0.365000,0.460000,0.288000
7,0.199500,0.743546,0.618000,0.364000,0.452000,0.296000
8,0.066500,0.784812,0.603000,0.352000,0.438000,0.284000
9,0.066500,0.849448,0.619000,0.361000,0.448000,0.287000
10,0.066500,0.838682,0.610000,0.349000,0.437000,0.279000


Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Tr

TrainOutput(global_step=555, training_loss=0.40411410621694616, metrics={'train_runtime': 2347.6778, 'train_samples_per_second': 1.898, 'train_steps_per_second': 0.236, 'total_flos': 9409568182370304.0, 'train_loss': 0.40411410621694616, 'epoch': 14.613333333333333})

## Step 6: Evaluation & Results

In [8]:

test_results = trainer.evaluate(tokenized_dataset['test'])
print("\nFinal Test Results:")
print(f"ROUGE-1: {test_results['eval_rouge1']:.3f}")
print(f"ROUGE-2: {test_results['eval_rouge2']:.3f}")
print(f"ROUGE-L: {test_results['eval_rougeL']:.3f}")
print(f"BLEU: {test_results['eval_bleu']:.3f}")

def generate_summary(text):
    """Production-ready inference function"""
    inputs = tokenizer(
        text,
        max_length=max_input_length,
        truncation=True,
        padding='max_length',
        return_tensors='pt'
    ).to(model.device)
    
    summary_ids = model.generate(
        inputs.input_ids,
        num_beams=6,
        length_penalty=1.2,
        max_length=max_target_length,
        early_stopping=True
    )
    
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

# Example usage
sample = dataset['test'][0]
print("\nInput Document:")
print(sample['structured_text'][:500] + "...")
print("\nGenerated Summary:")
print(generate_summary(sample['structured_text']))
print("\nReference Summary:")
print(sample['clean_summary'])

Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.



Final Test Results:
ROUGE-1: 0.619
ROUGE-2: 0.351
ROUGE-L: 0.432
BLEU: 0.291

Input Document:
<title>Cardiovascular Disease and Risk Factors in Asia A Selected Review</title>

<keywords>Cardiovascular disease prevention, Asia, stroke, coronary heart disease, risk factors, hypertension, salt intake, smoking.</keywords>

<abstract>Cardiovascular disease (CVD) prevention in Asia is an important issue for world health, because half of the worlds population lives in Asia. Asian countries and regions such as Japan, the Republic of Korea, the Peoples Republic of China, Hong Kong, Taiwan, and th...

Generated Summary:
Half of the world population lives in Asia, and Asian countries and regions have greater mortality and morbidity from stroke than from coronary heart disease (CHD). This is true even in Western countries, such as Japan and South Korea. The Seven Countries Study found that Japanese populations had lower fat intake, lower serum total cholesterol, and lower CHD than populations in 